In [1]:
import numpy as np
import nltk
from nltk.tokenize import wordpunct_tokenize
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import csv
from keras.models import Sequential 
from keras.layers import Dropout, Dense
from keras import callbacks
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def data_reader(filename:str):
    """Read the movie reviews with the given filename."""
    reviews = []
    rating = np.array(0)
    def parse(x):
        if x == 'positive':
            return 1
        else:
            return 0
    with open(filename, newline='') as csvfile:
        df = pd.read_csv(filename)
        df['sentiment'] = df['sentiment'].apply(lambda x: parse(x))
    return (df['review'].tolist(), df['sentiment'].to_numpy())

In [39]:
# test to see if method is working
# print(data_reader('IMDB Dataset.csv'))



In [24]:
def load_fast_text_embeddings(filename: str):
    """Loads the FastText embeddings from the file with the given filename."""
    
    dictionary = {}
    with open(filename) as reader:
        next(reader)
        for line in reader:
            line = line.strip()
            dictionary[line.split(' ')[0]] = np.array([float(x) for x in line.split(' ')[1:]])
    return (dictionary, len(dictionary[list(dictionary.keys())[0]]))

token2vector, dimensions =load_fast_text_embeddings(filename=embedding_path)
assert(dimensions == 300)

assert(token2vector['article'][0] == -0.1303 )

assert(token2vector['state'][2] ==0.098 )

In [25]:
def tokenize_sentences(sentences: [str]):
    """Tokenizes the given sentences"""
    sents = []
    for sentence in sentences:
        tokens = wordpunct_tokenize(sentence)
        sents.append([word.lower() for word in tokens if not word in nltk.corpus.stopwords.words('english') and word.isalpha()])
    return sents

In [35]:
def average(sentence):
    return np.array(np.array([token2vector.get(token, np.zeros(300)) for token in sentence])).mean(axis=0)

def map_to_vectors(tokenized_sentences: [[str]]):
    """Maps the given tokenized sentences to lists of vectors."""
    return np.array([average(sent) for sent in tokenized_sentences])

embedded_reviews = map_to_vectors(tokenized_reviews)

assert(len(embedded_reviews[0]) == 300)

assert(len(embedded_reviews[100]) == 300)




In [ ]:
# define a model
def MLP(input_shape: np.array):
    model = Sequential()
    model.add(Dense(100,activation='relu',input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(20,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    return model

In [26]:
mode = None
#______________________________________________
# uncomment by tutors
#mode = "grading"
#______________________________________________
if mode == "grading":
    data_path = '../dataset and embeddings/IMDB Dataset.csv'
    embedding_path  = '../dataset and embeddings/wiki-news-300d-10k.vec'
    model_path = '../model/best_model.ckpt'
else:
    # TODO
    data_path = 'IMDB Dataset.csv'
    embedding_path = 'embeddings/wiki-news-300d-10k.vec'
    model_path = None

In [27]:
reviews, sentiments = data_reader(data_path)
# convert numeric labels to one hot vectors
onehot_encoder = OneHotEncoder(sparse=False)
sentiments = sentiments.reshape(len(sentiments), 1)
sentiments = onehot_encoder.fit_transform(sentiments)

In [28]:
token2vector, dimensions = load_fast_text_embeddings(filename=embedding_path)

In [36]:
tokenized_reviews = tokenize_sentences(reviews)

KeyboardInterrupt: 

In [ ]:
embedded_reviews = map_to_vectors(tokenized_reviews)

In [ ]:
#TODO
# split the vectorized reviews into train-, and testset
train_test_split = 0.8
train_x = embedded_reviews[:len(embedded_reviews)*train_test_split]
train_y = sentiments[:len(embedded_reviews)*train_test_split]
test_x = embedded_reviews[len(embedded_reviews)*train_test_split:]
test_y = sentiments[len(embedded_reviews)*train_test_split:]
#ENDTODO

# train the model and save the best one on dev set
cp_callback = callbacks.ModelCheckpoint(filepath=model_path, verbose=1, save_weights_only=True, monitor='val_acc', save_best_only=True)
model = MLP(input_shape=(dimensions,))
# use the binary cross entropy to measure the errors
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y, epochs=4, batch_size=30, callbacks=[cp_callback], validation_split=0.2)

model.load_weights(model_path)
# calculate the accuracy on test set
_,acc = model.evaluate(test_x,test_y)
print('accuracy on test set:', acc)

predictions=model.predict(test_x)
# convert predictions to one hot vectors
predictions_oneHot = np.where(predictions > 0.5, 1, 0)
#print(predictions_oneHot)

In [ ]:
print('First Review')
print(tokenized_reviews[0])
print('Second Review')
print(tokenized_reviews[5])
print('Third Review')
print(tokenized_reviews[100])

(1 Point) point for implementing tokenize_sentences

In [ ]:
#______________________________________________
# should be written by tutors
point_for_c = 
#______________________________________________

(e)(4 Points) Check the functionMLP(input_shape:np.array)that defines our multi-layer perceptron (MLP).

(i)(1 Point) How many layers does this model have (including the input and the output layer)?
Answer:

5 layers

In [ ]:
#______________________________________________
# should be written by tutors
point_for_e_i = 
#______________________________________________

(ii) (1 Point) What is the size of the matrix that connects the input layer and the first hidden layer? 
Answer:

100

In [ ]:
#______________________________________________
# should be written by tutors
point_for_e_ii = 
#______________________________________________

(iii) (1 Point) How many units/neurons are in the output layer? Why? 
Answer:

2, because our categories are binary (negative, positive)

In [ ]:
#______________________________________________
# should be written by tutors
point_for_e_iii =
#______________________________________________

(iv) (1 Point)  What is the meaning of Dropout? 
Answer:

Dropout means that some of the input is randomly set to 0, in this case with a frequency of 0.2. This helps prevent overfitting.

In [ ]:
#______________________________________________
# should be written by tutors
point_for_e_iv = 
#______________________________________________

In [ ]:
# the following codes are used for grading, students can simply ignore them, but please don't change/delete them
# any modifications to the code are seen to be cheating
import csv
assertions = dict()
with open('../assertions.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        assertions[row['assertion sequence']] = row['content']

total_points = 0


try:  
    assert(assertions['a_1'] in reviews)
    assert(sentiments[reviews.index(assertions['a_1'])][1]==1)
    total_points += 1
except: 
    print('errors in your implementation for question (a)')

try:
    assert(total_points <= 1)
except AssertionError:
    print('errors in calculating the points for question (a)')


try:  
    assert(dimensions == int(assertions['b_1']))
    assert(token2vector['article'][0] == float(assertions['b_2']))
    assert(token2vector['state'][2] == float(assertions['b_3']))
    total_points += 1
except AssertionError:
    print('errors in your implementation for question (b)')


try:
    assert(total_points <= 2)
except AssertionError:
    print('errors in calculating the points for question (b)')
    

total_points += point_for_c 
try:
    assert(total_points <= 3)
except AssertionError:
    print('errors in calculating the points for question (c)')
    

try:
    assert(len(embedded_reviews[0]) == int(assertions['d_1']))
    assert(len(embedded_reviews[100]) == int(assertions['d_2']))
    assert(len(embedded_reviews[600]) == int(assertions['d_3']))
    total_points += 1
except AssertionError:
    print('errors in your implementation for question (d)')

try:
    assert(total_points <= 4)
except AssertionError:
    print('errors in calculating the points for question (d)')

points_for_first_four_questions = total_points



total_points += point_for_e_i 
try:
    assert(total_points <= 5)
except AssertionError:
    print('errors in calculating the points for question e(i)')


total_points += point_for_e_ii
try:
    assert(total_points <= 6)
except AssertionError:
    print('errors in calculating the points for question e(ii)')


total_points += point_for_e_iii 
try:
    assert(total_points <= 7)
except AssertionError:
    print('errors in calculating the points for question e(iii)')

 
total_points += point_for_e_iv 
try:
    assert(total_points <= 8)
except AssertionError:
    print('errors in calculating the points for question e(iv)')


point_for_f = 0
try:
    assert(len(train_x) > len(test_x))
    assert(len(train_y) > len(test_y))
    assert(len(train_x)+len(test_x)==len(embedded_reviews))
    assert(len(train_y)+len(test_y)==len(sentiments))
    point_for_f = 2
    total_points += point_for_f
except AssertionError:
    print('errors in your implementation for question (f)')

try:
    assert(total_points <= 10)
except AssertionError:
    print('errors in calculating the points for question (f)')

  

In [ ]:
print("points for the first four questions: ",points_for_first_four_questions)
print("point for e(i): ", point_for_e_i)
print("point for e(ii): ", point_for_e_ii)
print("point for e(iii): ", point_for_e_iii)
print("point for e(iv): ", point_for_e_iv)
print("point for f: ", point_for_f)
print("total points: ", total_points)

In [ ]:
# the following codes can be run if you want to save the result of all grading to a .csv file
# it can be run on the window system, but if you use colab, you need to at first locate to your current .ipynb file (cd xxx/xxx)
import os
# for windows users
filename = os.getcwd().split('\\')[-1]
#filename = os.getcwd().split('/')[-1]

In [ ]:
name = filename.split('_')[0]
matrikelnr= filename.split('_')[1]
# do you have any feedback
feedback = ""
#feedback += ' '.join([item for item in comments if len(item.strip())>0])
with open('../grading_for_HW9.csv','a+',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([name,matrikelnr,total_points,feedback])